In [7]:
import numpy as np 
import matplotlib.pyplot as plt 
import math 

#parameters of the model 
s = 100
T = 1
t = [0.25, 0.5, 0.75, 1]
k = 98
r = 0
q = 0.02

np.random.seed(50)
#constant volatility 
#np.random.seed(10)
sigma = 0.23
#in this case we can use a geometrical brownian motion to simulate the price at the fourth expiration 
delta = 0.25

#function to simulate the price using black scholes model
def simulate_price(s, n):
    pmat = np.empty((n,len(t)))
    for ind, j in enumerate(t):
        if ind == 0:
            bm = np.random.standard_normal(size = n)
            price = s*np.exp( delta*(r - q - 0.5*sigma**2 ) + sigma*np.sqrt(delta)*bm)
            pmat[:,ind] = np.squeeze(price)
        else:
            bm = np.random.standard_normal(size = n)
            pmat[:, ind ] = np.squeeze(np.exp( delta*(r - q - 0.5*sigma**2 ) + sigma*np.sqrt(delta)*bm))
            
    return pmat.cumprod(axis = 1)



In [8]:
#function to calculate the payoff of the option as stated in the assignemtn
def get_payoff(prices):
    payoff = np.maximum(np.mean(prices, axis = 1).reshape(prices.shape[0],1) - k,0)
    return payoff 

#regression function 
def regression(c, s, a, payoff):
    #build regression matrix 
    if np.all( s == a ):                                    #if they are the same, a and s, just take s
         x = np.c_[np.ones(c.shape[0]), s, s**2, s**3]        #this is done to avoid multicollinearity 
    else: 
         x = np.c_[np.ones(c.shape[0]), s, s**2, s**3, a,a**2, a**3]
            
    c = c.reshape(c.shape[0], 1)  
    
    beta = np.linalg.solve(x.T @ x, x.T @ c)    #beta of the regression 
    fitted = x @ beta                          #estimated continuation value
    exercise_today = (payoff >= fitted)        #I exercise today when the payoff is larger than the continuation value 
    return exercise_today


#function to calculate the continunation value
def continuation(j,cf):       
    #calculate the continuation value 
    #index to keep to get the continuation value (future cash flow)
    cf_ = cf[:,j+1:]
    discount = np.exp(  -delta* (np.arange(cf_.shape[1])+1) * r)
    return (cf_* discount).sum(axis = 1) 


#main function to calculate the price of the option 
def calculate_price(s,t, n):
    cash_flow = np.zeros((n,len(t)))        #cash flow matrix 
    prices = simulate_price(s, n)           #prices simulated 
    
    for j in reversed(range(0,len(t))):       #we do the for in the reverse order
        
        if j == len(t)-1:                     #if terminal time just fill the cash flow matrix 
            p_ = prices[:,0:j+1]         
            payoff = get_payoff(p_)
            cash_flow[:,j] = np.squeeze(payoff)
            
            
        else:                         
            
            p_ = prices[:,0:j+1]                  #restrict the price matrix
            payoff = get_payoff(p_)               #get the payoff calculated as mean over previous prices
            index = np.squeeze(payoff>0)  
                                                   #where is the payoff greater than 0? where do I have to take a decision ? 
            s = prices[index,j]                                           #s to use in the regression
            a = prices[index, 0:j+1].mean(axis = 1) 
           
            cont = continuation(j, cash_flow[index])   
            decision = regression(cont, s, a, payoff[index])                         #when I am better off by get money today 
            
            cash_flow[index,j] = np.squeeze(payoff[index]*(decision))                #update the cashflow matrix
            cash_flow[index, j+1:] = cash_flow[index, j+1:] * (~decision)    #update cashflow matrix 
     
    discount = np.exp(  -delta* (np.arange(cash_flow.shape[1])+1) * r)             

    price_vec = (discount * cash_flow).sum(axis = 1)
    return  price_vec.mean(), price_vec.std()/np.sqrt(n)


price, std = calculate_price(s, t, 1000000)
print(f"Price: {price}\nStandard Deviation of the Price: {std}")
        

Price: 6.962213917496692
Standard Deviation of the Price: 0.01023624376839333
